In [1]:
import torch
from transformers import  Pipeline,pipeline

/Users/tharhtet/.local/share/virtualenvs/rag-9Lk4wNMW/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def load_text_model(device_name):
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        torch_dtype=torch.bfloat16,
        
    
    )
    pipe.to(device_name)

    return pipe



In [8]:

system_prompt = """
Your name is AI bot and you are a helpful
chatbot responsible for teaching Machine Learning and AI to your users.
Always respond in markdown.
"""

def generate_text(pipe: Pipeline, prompt: str, temperature: float = 0.7) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ] 
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    predictions = pipe(
        prompt,
        temperature=temperature,
        max_new_tokens=256,
        do_sample=True,
        top_k=50,
        top_p=0.95,
    ) 
    output = predictions[0]["generated_text"].split("</s>\n<|assistant|>\n")[-1]
    return output

In [9]:
device_name  = None
if torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"
print("device_name : ",device_name)

device_name :  mps


In [10]:
prompt = "What is tensorflow"
pipe = load_text_model(device_name) 
output = generate_text(pipe, prompt)
output

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


AttributeError: 'TextGenerationPipeline' object has no attribute 'to'

### LLM model test

In [2]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="It's always great to meet you! I'm delighted that we can work together on a project. My name is [Your Name], and I'm a writer, editor, and proofreader with experience in various industries. Aside from my expertise, I also possess excellent communication skills and have worked for esteemed clients.\n\nMy goal is to provide quality content that meets your needs. Please let me know about any specific requirements you may have or the style of writing you prefer. I'm always open to discussing options to suit your project timelines and deadlines.\n\nI look forward to working with you on this assignment, and if there are any further details required, please let me know. Thank you for considering my services!", role='assistant', function_call=None, tool_calls=None)
